In [27]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import date
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from lxml import etree
import matplotlib.dates as mdates
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error, median_absolute_error, explained_variance_score
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from matplotlib.dates import AutoDateLocator, AutoDateFormatter
from scipy.stats import randint
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import layers, models, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
import graphviz
from IPython.display import Image
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.metrics import MeanSquaredError, MeanAbsoluteError
import psutil

In [ ]:
houses = {}
house_names = []
for i in range(1, 22):
    # number 14 is missing from the dataset
    if i == 14:
        continue
    file_name = f'CLEAN_House{i}.csv'
    try:
        # path_to_Houses.csv
        df = pd.read_csv(file_name, parse_dates=['Time'])
        # Add the dataframe to the list
        houses[file_name] = df
        #houses.append(df)
        house_names.append(file_name)
        print(file_name)
        print(houses[file_name].head())
    except FileNotFoundError:
        print(f'File not found: {file_name}')
        continue

In [ ]:
for name, df in houses.items():
    null_values = df.isnull().sum()
    print(null_values)
    print(name)

    #replace null values with the mean of their neighboring cells
    df['Aggregate'] = df['Aggregate'].fillna((df['Aggregate'].ffill() + df['Aggregate'].bfill()) / 2)

    df['Aggregate'].fillna(method='bfill', inplace=True)
    df['Aggregate'].fillna(method='ffill', inplace=True)


    df['Time'] = pd.to_datetime(df['Time'])
    df.set_index('Time', inplace=True)


    #total energy consumption
    df['Aggregate'].plot(figsize=(20, 10))
    plt.title('Total Energy Consumption Over Time')
    plt.ylabel('Energy (kWh)')
    plt.xlabel('Time')

    # Formatting date on x-axis
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m')) 
    plt.xticks(rotation = 90) 
    
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{name}.jpeg', format='jpeg', dpi=300)

    plt.show()
    

In [4]:
for name, df in houses.items():
    print(name)
    X = df.drop(['Aggregate', 'Time'], axis=1)
    y = df['Aggregate']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    regressor = RandomForestRegressor(n_estimators=20, random_state=42)
    regressor.fit(X_train, y_train)

    #predict on test set
    predictions = regressor.predict(X_test)

    #Evaluation
    rmse = mean_squared_error(y_test, predictions, squared=False)
    print(f'Root Mean Squared Error: {rmse}')


In [5]:
for name, df in houses.items():
    print(name)
    X = df.drop(['Aggregate', 'Time'], axis=1)
    y = df['Aggregate']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    print('RMSE:', mean_squared_error(y_test, predictions, squared=False))

In [6]:
for name, df in houses.items():
    print(name)
    model = GradientBoostingRegressor(n_estimators=20, random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    print('RMSE:', mean_squared_error(y_test, predictions, squared=False))


In [19]:
# Adapting the concepts of accuracy, precision, recall and confusion matrix by defining thresholds within which predictions are considered "accurate" or "correct." 
# This process is sometimes referred to as "binning" or "categorizing" continuous data to fit a classification model framework. 
def dynamic_regression_accuracy(y_true, y_pred, scale_factor=0.1):
    """Calculate accuracy with a dynamic tolerance based on a scale factor times the standard deviation of y_true."""
    tolerance = scale_factor * np.std(y_true)
    correct = np.abs(y_true - y_pred) <= tolerance
    return np.mean(correct)


def binarize_data(y, threshold):
    """Convert continuous data into binary classes based on a threshold."""
    return np.where(y > threshold, 1, 0)  # 1 for high consumption, 0 for low consumption

def calculate_precision_recall(y_true, y_pred, threshold):
    """Calculate precision for binarized data."""
    y_true_binarized = binarize_data(y_true, threshold)
    y_pred_binarized = binarize_data(y_pred, threshold)
    
    precision = precision_score(y_true_binarized, y_pred_binarized)

    
    return precision

def calculate_recall(y_true, y_pred, threshold):
    """Calculate recall for binarized data."""
    y_true_binarized = binarize_data(y_true, threshold)
    y_pred_binarized = binarize_data(y_pred, threshold)
    recall = recall_score(y_true_binarized, y_pred_binarized)
    return recall

def create_confusion_matrix(y_true, y_pred, threshold):
    """Create a confusion matrix for a given threshold for binarizing continuous outcomes."""
    bins = [0, threshold, np.inf]
    y_true_binned = np.digitize(y_true, bins) - 1  # Convert to 0 or 1
    y_pred_binned = np.digitize(y_pred, bins) - 1
    cm = confusion_matrix(y_true_binned, y_pred_binned)
    return cm

def custom_mape(y_true, y_pred):
    epsilon = 1e-8
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.clip_by_value(tf.cast(y_pred, tf.float32), 0, float('inf'))  # Clip predictions at zero
    
    mape = tf.reduce_mean(tf.abs((y_true - y_pred) / (tf.maximum(tf.abs(y_true), epsilon)))) * 100
    return mape




In [28]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, SimpleRNN, LSTM, Bidirectional, GRU, Conv1D, MaxPooling1D, Flatten, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, TimeDistributed, Conv2D, Concatenate, Attention
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dropout, Dense, Input, GlobalAveragePooling1D, Add, Conv1D, MaxPooling1D, Bidirectional, LSTM, Attention, GRU, LeakyReLU, BatchNormalization 
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import optuna
from optuna.integration import TFKerasPruningCallback
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error, explained_variance_score, mean_squared_log_error
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour
from optuna.visualization import plot_parallel_coordinate
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dropout, Dense, Input, GlobalAveragePooling1D, Add, Conv1D, MaxPooling1D, Bidirectional, GRU, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import RobustScaler


# Define RNN model
def create_rnn_model(input_shape, optimizer='adam', activation = 'relu', learning_rate=0.001):
    inputs = Input(shape=input_shape)
    x = SimpleRNN(32, activation=activation, return_sequences=True)(inputs)
    x = Dropout(0.3)(x)
    x = SimpleRNN(16, activation=activation, return_sequences=False)(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=tf.losses.Huber(delta=0.5), metrics=['mse', 'mae'])
    return model

# Define LSTM model
def create_lstm_model(input_shape, optimizer='adam', activation = 'relu', learning_rate=0.001):
    inputs = Input(shape=input_shape)
    x = LSTM(32, activation=activation, return_sequences=True)(inputs)
    x = Dropout(0.1)(x)
    x = LSTM(16, activation=activation)(inputs)
    x = Dropout(0.1)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=tf.losses.Huber(delta=0.5), metrics=['mse', 'mae'])
    return model

# Define BiLSTM model
def create_bilstm_model(input_shape, optimizer='adam', activation = 'relu', learning_rate=0.001):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(32, activation=activation))(inputs)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=tf.losses.Huber(delta=0.5), metrics=['mse', 'mae'])
    return model

# Define GRU model
def create_gru_model(input_shape, optimizer='adam', activation = 'relu', learning_rate=0.001):
    inputs = Input(shape=input_shape)
    x = GRU(32, activation=activation, return_sequences=True)(inputs)
    x = Dropout(0.1)(x)
    x = GRU(16, activation=activation)(inputs)
    x = Dropout(0.1)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=tf.losses.Huber(delta=0.5), metrics=['mse', 'mae'])
    return model

# Define BiGRU model
def create_bigru_model(input_shape, optimizer='adam', activation = 'relu', learning_rate=0.001):
    inputs = Input(shape=input_shape)
    x = Bidirectional(GRU(32, activation=activation))(inputs)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=tf.losses.Huber(delta=0.5), metrics=['mse', 'mae'])
    return model


# Transformer Encoder Block
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1):
    # Multi-Head Self-Attention
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = Dropout(dropout)(x)
    x = Add()([x, inputs])  # Residual connection
    x = LayerNormalization(epsilon=1e-6)(x)

    # Feed Forward Network
    x_ff = Dense(ff_dim, activation=LeakyReLU())(x)
    x_ff = Dropout(dropout)(x_ff)
    x_ff = Dense(inputs.shape[-1])(x_ff)
    x = Add()([x, x_ff])  # Residual connection
    x = LayerNormalization(epsilon=1e-6)(x)

    return x


# Model Definition
def cnn_lstm_attention_transformer_model(input_shape, num_cnn_filters=16, kernel_size=2, lstm_units=32, gru_units=32, head_size=16, num_heads=2, ff_dim=16, num_transformer_blocks=3, dropout=0.2, weight_decay=1e-6):
    inputs = Input(shape=input_shape)
    
    x = Conv1D(filters=num_cnn_filters, kernel_size=kernel_size, padding='same', activation='relu', kernel_regularizer=l2(weight_decay))(inputs)
    x = MaxPooling1D(pool_size=2)(x) 
    x = Conv1D(filters=num_cnn_filters, kernel_size=kernel_size, padding='same', activation='relu', kernel_regularizer=l2(weight_decay))(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(filters=num_cnn_filters * 2, kernel_size=kernel_size, padding='same', activation='relu', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(dropout)(x)  # Dropout after pooling
    
    x = LayerNormalization()(x)

    x = Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = Bidirectional(LSTM(lstm_units // 2, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = LayerNormalization()(x)

    x = Bidirectional(GRU(gru_units, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = Bidirectional(GRU(gru_units, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = Bidirectional(GRU(gru_units // 2, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = LayerNormalization()(x)

    # Attention Mechanism (optional)
    attention = Attention()([x, x])  # Self-attention over the LSTM/GRU output
    x = Add()([x, attention])  # Add residual connection after attention
    
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size=head_size, num_heads=num_heads, ff_dim=ff_dim, dropout=dropout)

    x = GlobalAveragePooling1D()(x)
    x = Dropout(dropout)(x)  # Dropout after pooling layer
    outputs = Dense(1, kernel_regularizer=l2(weight_decay))(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss= tf.losses.Huber(delta=1.5), metrics=['mse', 'mae'])
    
    return model

def create_features(df):
    df['Hour'] = df.index.hour
    df['Day_of_Week'] = df.index.dayofweek
    df['Month'] = df.index.month
    df['Lag_1'] = df['Aggregate'].shift(1)
    df['Lag_450'] = df['Aggregate'].shift(450)
    df['Lag_5400'] = df['Aggregate'].shift(5400)
    df['Lag_10800'] = df['Aggregate'].shift(10800)
    df.dropna(inplace=True)
    return df

def print_cpu_usage(phase):
    cpu_percent = psutil.cpu_percent(interval=1)
    print(f"CPU usage during {phase}: {cpu_percent}%")
    

def create_sequences_with_additional_features(dataset, additional_features, window_size=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - window_size):
        seq_X = dataset[i:(i + window_size), :]
        seq_features = additional_features[i + window_size - 1]  # Add the corresponding additional feature to each sequence
        dataX.append(np.concatenate((seq_X, np.tile(seq_features, (window_size, 1))), axis=1))
        dataY.append(dataset[i + window_size, :])
    return np.array(dataX), np.array(dataY)

def prepare_timeseries_generator(data, features, window_size=24, batch_size=64):
    generator_input = np.concatenate([features, data], axis=1)  # Combine features and target for input
    generator = TimeseriesGenerator(generator_input, data, length=window_size, batch_size=batch_size)
    return generator


In [ ]:
RNN_models={}
tf.get_logger().setLevel('ERROR')
for name, df in houses.items():
    print(name)
    
    df = create_features(df)
    
    # Step 1: Proper Scaling
    aggregate = df['Aggregate'].values.reshape(-1, 1)
    additional_features = df[['Hour', 'Day_of_Week', 'Month', 'Lag_1', 'Lag_450', 'Lag_5400', 'Lag_10800']].values

    # Scale after splitting to avoid leakage
    split_point = int(len(aggregate) * 0.80)
    
    scaler_aggregate = RobustScaler()
    train_aggregate = scaler_aggregate.fit_transform(aggregate[:split_point])
    test_aggregate = scaler_aggregate.transform(aggregate[split_point:])

    scaler_additional = RobustScaler()
    train_additional_features = scaler_additional.fit_transform(additional_features[:split_point])
    test_additional_features = scaler_additional.transform(additional_features[split_point:])

    # Step 5: Investigate Overfitting (Balanced Split)
    window_size = 24
    batch_size = 64
    train_generator = prepare_timeseries_generator(train_aggregate, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator = prepare_timeseries_generator(test_aggregate, test_additional_features, window_size=window_size, batch_size=batch_size)

    input_shape = (window_size, train_generator[0][0].shape[2])

    model = create_rnn_model(input_shape=input_shape)

    early_stopping = EarlyStopping(monitor='loss', patience=2, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-3)
    print_cpu_usage("Initialization")
    history = model.fit(train_generator, epochs=30, validation_data=test_generator, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)
    print_cpu_usage("after training")
    
    print_cpu_usage("before prediction")
    predictions = model.predict(test_generator)
    actual = test_aggregate[window_size:]  #Align actuals &  predictions

    actual = scaler_aggregate.inverse_transform(actual)
    predictions = scaler_aggregate.inverse_transform(predictions)
    print_cpu_usage("after prediction")

    #  metrics
    rmse = mean_squared_error(actual, predictions, squared = False)
    mae = mean_absolute_error(actual, predictions)
    r2 = r2_score(actual, predictions)
    mse = mean_squared_error(actual, predictions)
    
    # Extra classification metrics for curiosity 
    epsilon = 1e-5
    mape = np.mean(np.abs((actual - predictions) / np.maximum(np.abs(actual), epsilon))) * 100
    median_ae = median_absolute_error(actual, predictions)
    explained_variance = explained_variance_score(actual, predictions)
    mbd = np.mean(predictions - actual)
    try:
        msle = mean_squared_log_error(actual, predictions) 
        # most ofter will be 0
    except ValueError as e:
        print("Error calculating MSLE:", e)
        msle = None  # Handle cases where MSLE calculation fails
    

    #If we choose to Convert to a Classification Problem, it would be also wise to have these metrics:
    threshold = np.mean(actual)  # Example threshold
    print(f'Threshold: {threshold}')
    accuracy = dynamic_regression_accuracy(actual, predictions, threshold) 
    precision = calculate_precision_recall(actual, predictions, threshold)
    recall = calculate_recall(actual, predictions, threshold)
    conf_matrix = create_confusion_matrix(actual, predictions, threshold)
    
    
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R²: {r2}")
    print(f'MSE: {mse}')
    print(f'MAPE: {mape}%')
    print(f'Median Absolute Error: {median_ae}')
    print(f'Explained Variance Score: {explained_variance}')
    print(f'MSLE: {msle if msle is not None else "Calculation failed due to negative or zero values"}')
    print(f'Mean Bias Deviation: {mbd}')
    print(f'Accuracy: {accuracy}')
    print(f'Confusion Matrix:\n{conf_matrix}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    model_weights_path = f"{name}_RNN_weights.h5"
    model.save_weights(model_weights_path)
    
    RNN_models[name] = {
        'model': model,
        'actuals': actual,
        'predictions': predictions,
        'model_weights': model_weights_path,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mse': mse,
        'mape': mape,
        'median_ae': median_ae,
        'explained_variance': explained_variance,
        'msle': msle,
        'mbd': mbd,
        'accuracy': accuracy,
        'conf_matrix': conf_matrix,
        'precision': precision,
        'recall': recall
    }
    points_in_24h = int((24 * 3600) / 7)
    # Plot results
    plt.figure(figsize=(20, 10))
    aligned_dates = df.index[split_point + window_size:split_point + window_size + len(actual)]
    aligned_dates_24h = aligned_dates[:points_in_24h]
    
    #store the results
    results_df = pd.DataFrame({
        'Date': aligned_dates,
        'Actual': actual.flatten(),
        'Predicted': predictions.flatten()
    })

    results_df.to_csv(f"RNN_{name}_results.csv", index=False)
    print(f"Results saved to RNN_{name}_results.csv")
    #for  entire test period
    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual, label='Actual')
    plt.plot(aligned_dates, predictions, label='Predicted', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption')
    plt.title(f'Actual vs Predicted for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"RNN_House_{name}_full_results.jpeg",format='jpeg', dpi=300 ) 
    plt.show()

    #for the first day
    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual[:points_in_24h], label='Actual')
    plt.plot(aligned_dates_24h, predictions[:points_in_24h], label='Predicted', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption')
    plt.title(f'Actual vs Predicted for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)   
    plt.savefig(f"RNN_House_{name}_24h_results.jpeg", format='jpeg', dpi=300)  
    plt.show()

In [ ]:
LSTM_models={}
tf.get_logger().setLevel('ERROR')
for name, df in houses.items():
    print(name)
    
    df = create_features(df)
    
    # Step 1: Proper Scaling
    aggregate = df['Aggregate'].values.reshape(-1, 1)
    additional_features = df[['Hour', 'Day_of_Week', 'Month', 'Lag_1', 'Lag_450', 'Lag_5400', 'Lag_10800']].values

    # Scale after splitting to avoid leakage
    split_point = int(len(aggregate) * 0.80)
    
    scaler_aggregate = RobustScaler()
    train_aggregate = scaler_aggregate.fit_transform(aggregate[:split_point])
    test_aggregate = scaler_aggregate.transform(aggregate[split_point:])

    scaler_additional = RobustScaler()
    train_additional_features = scaler_additional.fit_transform(additional_features[:split_point])
    test_additional_features = scaler_additional.transform(additional_features[split_point:])

    # Step 5: Investigate Overfitting (Balanced Split)
    window_size = 24
    batch_size = 64
    train_generator = prepare_timeseries_generator(train_aggregate, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator = prepare_timeseries_generator(test_aggregate, test_additional_features, window_size=window_size, batch_size=batch_size)

    input_shape = (window_size, train_generator[0][0].shape[2])

    model = create_lstm_model(input_shape=input_shape)

    early_stopping = EarlyStopping(monitor='loss', patience=2, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-3)
    print_cpu_usage("Initialization")
    history = model.fit(train_generator, epochs=30, validation_data=test_generator, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)
    print_cpu_usage("after training")
    
    print_cpu_usage("before prediction")
    predictions = model.predict(test_generator)
    actual = test_aggregate[window_size:]  #Align actuals &  predictions

    actual = scaler_aggregate.inverse_transform(actual)
    predictions = scaler_aggregate.inverse_transform(predictions)
    print_cpu_usage("after prediction")

    #  metrics
    rmse = mean_squared_error(actual, predictions, squared = False)
    mae = mean_absolute_error(actual, predictions)
    r2 = r2_score(actual, predictions)
    mse = mean_squared_error(actual, predictions)
    
    # Extra classification metrics for curiosity 
    epsilon = 1e-5
    mape = np.mean(np.abs((actual - predictions) / np.maximum(np.abs(actual), epsilon))) * 100
    median_ae = median_absolute_error(actual, predictions)
    explained_variance = explained_variance_score(actual, predictions)
    mbd = np.mean(predictions - actual)
    try:
        msle = mean_squared_log_error(actual, predictions) 
        # most ofter will be 0
    except ValueError as e:
        print("Error calculating MSLE:", e)
        msle = None  # Handle cases where MSLE calculation fails
    

    #If we choose to Convert to a Classification Problem, it would be also wise to have these metrics:
    threshold = np.mean(actual)  # Example threshold
    print(f'Threshold: {threshold}')
    accuracy = dynamic_regression_accuracy(actual, predictions, threshold) 
    precision = calculate_precision_recall(actual, predictions, threshold)
    recall = calculate_recall(actual, predictions, threshold)
    conf_matrix = create_confusion_matrix(actual, predictions, threshold)
    
    
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R²: {r2}")
    print(f'MSE: {mse}')
    print(f'MAPE: {mape}%')
    print(f'Median Absolute Error: {median_ae}')
    print(f'Explained Variance Score: {explained_variance}')
    print(f'MSLE: {msle if msle is not None else "Calculation failed due to negative or zero values"}')
    print(f'Mean Bias Deviation: {mbd}')
    print(f'Accuracy: {accuracy}')
    print(f'Confusion Matrix:\n{conf_matrix}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    model_weights_path = f"{name}_LSTM_weights.h5"
    model.save_weights(model_weights_path)
    
    LSTM_models[name] = {
        'model': model,
        'actuals': actual,
        'predictions': predictions,
        'model_weights': model_weights_path,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mse': mse,
        'mape': mape,
        'median_ae': median_ae,
        'explained_variance': explained_variance,
        'msle': msle,
        'mbd': mbd,
        'accuracy': accuracy,
        'conf_matrix': conf_matrix,
        'precision': precision,
        'recall': recall
    }
    points_in_24h = int((24 * 3600) / 7)
    # Plot results
    plt.figure(figsize=(20, 10))
    aligned_dates = df.index[split_point + window_size:split_point + window_size + len(actual)]
    aligned_dates_24h = aligned_dates[:points_in_24h]
    
    #store the results
    results_df = pd.DataFrame({
        'Date': aligned_dates,
        'Actual': actual.flatten(),
        'Predicted': predictions.flatten()
    })

    results_df.to_csv(f"LSTM_{name}_results.csv", index=False)
    print(f"Results saved to LSTM_{name}_results.csv")
    #for  entire test period
    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual, label='Actual')
    plt.plot(aligned_dates, predictions, label='Predicted', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption')
    plt.title(f'Actual vs Predicted for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"LSTM_House_{name}_full_results.jpeg",format='jpeg', dpi=300 ) 
    plt.show()

    #for the first day
    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual[:points_in_24h], label='Actual')
    plt.plot(aligned_dates_24h, predictions[:points_in_24h], label='Predicted', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption')
    plt.title(f'Actual vs Predicted for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)   
    plt.savefig(f"LSTM_House_{name}_24h_results.jpeg", format='jpeg', dpi=300)  
    plt.show()

In [ ]:
GRU_models={}
tf.get_logger().setLevel('ERROR')
for name, df in houses.items():
    print(name)
    
    df = create_features(df)
    
    # Step 1: Proper Scaling
    aggregate = df['Aggregate'].values.reshape(-1, 1)
    additional_features = df[['Hour', 'Day_of_Week', 'Month', 'Lag_1', 'Lag_450', 'Lag_5400', 'Lag_10800']].values

    # Scale after splitting to avoid leakage
    split_point = int(len(aggregate) * 0.80)
    
    scaler_aggregate = RobustScaler()
    train_aggregate = scaler_aggregate.fit_transform(aggregate[:split_point])
    test_aggregate = scaler_aggregate.transform(aggregate[split_point:])

    scaler_additional = RobustScaler()
    train_additional_features = scaler_additional.fit_transform(additional_features[:split_point])
    test_additional_features = scaler_additional.transform(additional_features[split_point:])

    # Step 5: Investigate Overfitting (Balanced Split)
    window_size = 24
    batch_size = 64
    train_generator = prepare_timeseries_generator(train_aggregate, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator = prepare_timeseries_generator(test_aggregate, test_additional_features, window_size=window_size, batch_size=batch_size)

    input_shape = (window_size, train_generator[0][0].shape[2])

    model = create_gru_model(input_shape=input_shape)

    early_stopping = EarlyStopping(monitor='loss', patience=2, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-3)
    print_cpu_usage("Initialization")
    history = model.fit(train_generator, epochs=30, validation_data=test_generator, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)
    print_cpu_usage("after training")
    
    print_cpu_usage("before prediction")
    predictions = model.predict(test_generator)
    actual = test_aggregate[window_size:]  #Align actuals &  predictions

    actual = scaler_aggregate.inverse_transform(actual)
    predictions = scaler_aggregate.inverse_transform(predictions)
    print_cpu_usage("after prediction")

    #  metrics
    rmse = mean_squared_error(actual, predictions, squared = False)
    mae = mean_absolute_error(actual, predictions)
    r2 = r2_score(actual, predictions)
    mse = mean_squared_error(actual, predictions)
    
    # Extra classification metrics for curiosity 
    epsilon = 1e-5
    mape = np.mean(np.abs((actual - predictions) / np.maximum(np.abs(actual), epsilon))) * 100
    median_ae = median_absolute_error(actual, predictions)
    explained_variance = explained_variance_score(actual, predictions)
    mbd = np.mean(predictions - actual)
    try:
        msle = mean_squared_log_error(actual, predictions) 
        # most ofter will be 0
    except ValueError as e:
        print("Error calculating MSLE:", e)
        msle = None  # Handle cases where MSLE calculation fails
    

    #If we choose to Convert to a Classification Problem, it would be also wise to have these metrics:
    threshold = np.mean(actual)  # Example threshold
    print(f'Threshold: {threshold}')
    accuracy = dynamic_regression_accuracy(actual, predictions, threshold) 
    precision = calculate_precision_recall(actual, predictions, threshold)
    recall = calculate_recall(actual, predictions, threshold)
    conf_matrix = create_confusion_matrix(actual, predictions, threshold)
    
    
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R²: {r2}")
    print(f'MSE: {mse}')
    print(f'MAPE: {mape}%')
    print(f'Median Absolute Error: {median_ae}')
    print(f'Explained Variance Score: {explained_variance}')
    print(f'MSLE: {msle if msle is not None else "Calculation failed due to negative or zero values"}')
    print(f'Mean Bias Deviation: {mbd}')
    print(f'Accuracy: {accuracy}')
    print(f'Confusion Matrix:\n{conf_matrix}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    model_weights_path = f"{name}_GRU_weights.h5"
    model.save_weights(model_weights_path)
    
    GRU_models[name] = {
        'model': model,
        'actuals': actual,
        'predictions': predictions,
        'model_weights': model_weights_path,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mse': mse,
        'mape': mape,
        'median_ae': median_ae,
        'explained_variance': explained_variance,
        'msle': msle,
        'mbd': mbd,
        'accuracy': accuracy,
        'conf_matrix': conf_matrix,
        'precision': precision,
        'recall': recall
    }
    points_in_24h = int((24 * 3600) / 7)
    # Plot results
    plt.figure(figsize=(20, 10))
    aligned_dates = df.index[split_point + window_size:split_point + window_size + len(actual)]
    aligned_dates_24h = aligned_dates[:points_in_24h]
    
    #store the results
    results_df = pd.DataFrame({
        'Date': aligned_dates,
        'Actual': actual.flatten(),
        'Predicted': predictions.flatten()
    })

    results_df.to_csv(f"GRU_{name}_results.csv", index=False)
    print(f"Results saved to GRU_{name}_results.csv")
    #for  entire test period
    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual, label='Actual')
    plt.plot(aligned_dates, predictions, label='Predicted', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption')
    plt.title(f'Actual vs Predicted for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"GRU_House_{name}_full_results.jpeg",format='jpeg', dpi=300 ) 
    plt.show()

    #for the first day
    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual[:points_in_24h], label='Actual')
    plt.plot(aligned_dates_24h, predictions[:points_in_24h], label='Predicted', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption')
    plt.title(f'Actual vs Predicted for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)   
    plt.savefig(f"GRU_House_{name}_24h_results.jpeg", format='jpeg', dpi=300)  
    plt.show()

In [ ]:
BiLSTM_models={}
tf.get_logger().setLevel('ERROR')
for name, df in houses.items():
    print(name)
    
    df = create_features(df)
    
    # Step 1: Proper Scaling
    aggregate = df['Aggregate'].values.reshape(-1, 1)
    additional_features = df[['Hour', 'Day_of_Week', 'Month', 'Lag_1', 'Lag_450', 'Lag_5400', 'Lag_10800']].values

    # Scale after splitting to avoid leakage
    split_point = int(len(aggregate) * 0.80)
    
    scaler_aggregate = RobustScaler()
    train_aggregate = scaler_aggregate.fit_transform(aggregate[:split_point])
    test_aggregate = scaler_aggregate.transform(aggregate[split_point:])

    scaler_additional = RobustScaler()
    train_additional_features = scaler_additional.fit_transform(additional_features[:split_point])
    test_additional_features = scaler_additional.transform(additional_features[split_point:])

    # Step 5: Investigate Overfitting (Balanced Split)
    window_size = 24
    batch_size = 64
    train_generator = prepare_timeseries_generator(train_aggregate, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator = prepare_timeseries_generator(test_aggregate, test_additional_features, window_size=window_size, batch_size=batch_size)

    input_shape = (window_size, train_generator[0][0].shape[2])

    model = create_bilstm_model(input_shape=input_shape)

    early_stopping = EarlyStopping(monitor='loss', patience=2, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-3)
    print_cpu_usage("Initialization")
    history = model.fit(train_generator, epochs=30, validation_data=test_generator, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)
    print_cpu_usage("after training")
    
    print_cpu_usage("before prediction")
    predictions = model.predict(test_generator)
    actual = test_aggregate[window_size:]  #Align actuals &  predictions

    actual = scaler_aggregate.inverse_transform(actual)
    predictions = scaler_aggregate.inverse_transform(predictions)
    print_cpu_usage("after prediction")

    #  metrics
    rmse = mean_squared_error(actual, predictions, squared = False)
    mae = mean_absolute_error(actual, predictions)
    r2 = r2_score(actual, predictions)
    mse = mean_squared_error(actual, predictions)
    
    # Extra classification metrics for curiosity 
    epsilon = 1e-5
    mape = np.mean(np.abs((actual - predictions) / np.maximum(np.abs(actual), epsilon))) * 100
    median_ae = median_absolute_error(actual, predictions)
    explained_variance = explained_variance_score(actual, predictions)
    mbd = np.mean(predictions - actual)
    try:
        msle = mean_squared_log_error(actual, predictions) 
        # most ofter will be 0
    except ValueError as e:
        print("Error calculating MSLE:", e)
        msle = None  # Handle cases where MSLE calculation fails
    

    #If we choose to Convert to a Classification Problem, it would be also wise to have these metrics:
    threshold = np.mean(actual)  # Example threshold
    print(f'Threshold: {threshold}')
    accuracy = dynamic_regression_accuracy(actual, predictions, threshold) 
    precision = calculate_precision_recall(actual, predictions, threshold)
    recall = calculate_recall(actual, predictions, threshold)
    conf_matrix = create_confusion_matrix(actual, predictions, threshold)
    
    
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R²: {r2}")
    print(f'MSE: {mse}')
    print(f'MAPE: {mape}%')
    print(f'Median Absolute Error: {median_ae}')
    print(f'Explained Variance Score: {explained_variance}')
    print(f'MSLE: {msle if msle is not None else "Calculation failed due to negative or zero values"}')
    print(f'Mean Bias Deviation: {mbd}')
    print(f'Accuracy: {accuracy}')
    print(f'Confusion Matrix:\n{conf_matrix}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    model_weights_path = f"{name}_BiLSTM_weights.h5"
    model.save_weights(model_weights_path)
    
    BiLSTM_models[name] = {
        'model': model,
        'actuals': actual,
        'predictions': predictions,
        'model_weights': model_weights_path,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mse': mse,
        'mape': mape,
        'median_ae': median_ae,
        'explained_variance': explained_variance,
        'msle': msle,
        'mbd': mbd,
        'accuracy': accuracy,
        'conf_matrix': conf_matrix,
        'precision': precision,
        'recall': recall
    }
    points_in_24h = int((24 * 3600) / 7)
    # Plot results
    plt.figure(figsize=(20, 10))
    aligned_dates = df.index[split_point + window_size:split_point + window_size + len(actual)]
    aligned_dates_24h = aligned_dates[:points_in_24h]
    
    #store the results
    results_df = pd.DataFrame({
        'Date': aligned_dates,
        'Actual': actual.flatten(),
        'Predicted': predictions.flatten()
    })

    results_df.to_csv(f"BiLSTM_{name}_results.csv", index=False)
    print(f"Results saved to BiLSTM_{name}_results.csv")
    #for  entire test period
    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual, label='Actual')
    plt.plot(aligned_dates, predictions, label='Predicted', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption')
    plt.title(f'Actual vs Predicted for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"BiLSTM{name}_full_results.jpeg",format='jpeg', dpi=300 ) 
    plt.show()

    #for the first day
    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual[:points_in_24h], label='Actual')
    plt.plot(aligned_dates_24h, predictions[:points_in_24h], label='Predicted', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption')
    plt.title(f'Actual vs Predicted for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)   
    plt.savefig(f"BiLSTM_{name}_24h_results.jpeg", format='jpeg', dpi=300)  
    plt.show()

In [ ]:
Transformer_models={}
tf.get_logger().setLevel('ERROR')
for name, df in houses.items():
    print(name)
    
    df = create_features(df)
    
    # Step 1: Proper Scaling
    aggregate = df['Aggregate'].values.reshape(-1, 1)
    additional_features = df[['Hour', 'Day_of_Week', 'Month', 'Lag_1', 'Lag_450', 'Lag_5400', 'Lag_10800']].values

    # Scale after splitting to avoid leakage
    split_point = int(len(aggregate) * 0.80)
    
    scaler_aggregate = RobustScaler()
    train_aggregate = scaler_aggregate.fit_transform(aggregate[:split_point])
    test_aggregate = scaler_aggregate.transform(aggregate[split_point:])

    scaler_additional = RobustScaler()
    train_additional_features = scaler_additional.fit_transform(additional_features[:split_point])
    test_additional_features = scaler_additional.transform(additional_features[split_point:])

    # Step 5: Investigate Overfitting (Balanced Split)
    window_size = 24
    batch_size = 64
    train_generator = prepare_timeseries_generator(train_aggregate, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator = prepare_timeseries_generator(test_aggregate, test_additional_features, window_size=window_size, batch_size=batch_size)

    input_shape = (window_size, train_generator[0][0].shape[2])

    model = cnn_lstm_attention_transformer_model(input_shape=input_shape, dropout=0.2, weight_decay=1e-7)

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)
    print_cpu_usage("Initialization")
    history = model.fit(train_generator, epochs=30, validation_data=test_generator, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)
    print_cpu_usage("after training")
    
    print_cpu_usage("before prediction")
    predictions = model.predict(test_generator)
    actual = test_aggregate[window_size:]  #Align actuals &  predictions

    actual = scaler_aggregate.inverse_transform(actual)
    predictions = scaler_aggregate.inverse_transform(predictions)
    print_cpu_usage("after prediction")

    #  metrics
    rmse = mean_squared_error(actual, predictions, squared = False)
    mae = mean_absolute_error(actual, predictions)
    r2 = r2_score(actual, predictions)
    mse = mean_squared_error(actual, predictions)
    
    # Extra classification metrics for curiosity 
    epsilon = 1e-5
    mape = np.mean(np.abs((actual - predictions) / np.maximum(np.abs(actual), epsilon))) * 100
    median_ae = median_absolute_error(actual, predictions)
    explained_variance = explained_variance_score(actual, predictions)
    mbd = np.mean(predictions - actual)
    try:
        msle = mean_squared_log_error(actual, predictions) 
        # most ofter will be 0
    except ValueError as e:
        print("Error calculating MSLE:", e)
        msle = None  # Handle cases where MSLE calculation fails
    

    #If we choose to Convert to a Classification Problem, it would be also wise to have these metrics:
    threshold = np.mean(actual)  # Example threshold
    print(f'Threshold: {threshold}')
    accuracy = dynamic_regression_accuracy(actual, predictions, threshold) 
    precision = calculate_precision_recall(actual, predictions, threshold)
    recall = calculate_recall(actual, predictions, threshold)
    conf_matrix = create_confusion_matrix(actual, predictions, threshold)
    
    
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R²: {r2}")
    print(f'MSE: {mse}')
    print(f'MAPE: {mape}%')
    print(f'Median Absolute Error: {median_ae}')
    print(f'Explained Variance Score: {explained_variance}')
    print(f'MSLE: {msle if msle is not None else "Calculation failed due to negative or zero values"}')
    print(f'Mean Bias Deviation: {mbd}')
    print(f'Accuracy: {accuracy}')
    print(f'Confusion Matrix:\n{conf_matrix}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    model_weights_path = f"{name}_Transformers_weights.h5"
    model.save_weights(model_weights_path)
    
    Transformer_models[name] = {
        'model': model,
        'actuals': actual,
        'predictions': predictions,
        'model_weights': model_weights_path,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mse': mse,
        'mape': mape,
        'median_ae': median_ae,
        'explained_variance': explained_variance,
        'msle': msle,
        'mbd': mbd,
        'accuracy': accuracy,
        'conf_matrix': conf_matrix,
        'precision': precision,
        'recall': recall
    }
    points_in_24h = int((24 * 3600) / 7)
    # Plot results
    plt.figure(figsize=(20, 10))
    aligned_dates = df.index[split_point + window_size:split_point + window_size + len(actual)]
    aligned_dates_24h = aligned_dates[:points_in_24h]
    
    #store the results
    results_df = pd.DataFrame({
        'Date': aligned_dates,
        'Actual': actual.flatten(),
        'Predicted': predictions.flatten()
    })

    results_df.to_csv(f"Hybrid_{name}_results.csv", index=False)
    print(f"Results saved to Hybrid_{name}_results.csv")
    #for  entire test period
    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual, label='Actual')
    plt.plot(aligned_dates, predictions, label='Predicted', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption')
    plt.title(f'Actual vs Predicted for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"Hybrid_House_{name}_full_results.jpeg",format='jpeg', dpi=300 ) 
    plt.show()

    #for the first day
    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual[:points_in_24h], label='Actual')
    plt.plot(aligned_dates_24h, predictions[:points_in_24h], label='Predicted', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption')
    plt.title(f'Actual vs Predicted for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)   
    plt.savefig(f"Hybrid_House_{name}_24h_results.jpeg", format='jpeg', dpi=300)  
    plt.show()